## 네이버 뉴스 링크 수집

### 함수 생성

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests
import json
from bs4 import BeautifulSoup as BTS
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import os

In [ ]:
# 일간지 기사에서 네이버뉴스 링크를 수집하고 데이터프레임으로 반환하는 함수를 생성합니다.
def NaverNewsLink(searchWord, period = None, bgnDate = None, endDate = None, startNo = 1):

    # 조회 구분을 설정합니다.
    if period is not None:
        period = 3  # 0: 전체, 4: 1일, 1: 1주, 2: 1개월, 13: 3개월, 6: 6개월, 5: 1년, 3: 직접입력
    else:
        period = 0
        
    # HTTP 요청을 실행합니다.
    res = requests.get(
        url = 'https://s.search.naver.com/p/newssearch/2/search.naver', 
        params = {
            'office_category': 1,  # 0: 전체, 1: 일간지, 2: 방송/통신, 3: 경제/IT, 4: 인터넷신문, 5: 스포츠/연예, 6: 지역지, 7: 매거진, 8: 전문지/기타
            'office_type': 3,  # 0: 전체, 1: 지역언론사별, 2: 가나다순, 3: 언론사 분류순
            'pd': period,
            'ds': bgnDate, 
            'de': endDate, 
            'query': searchWord,
            'rev': 31,
            'service_area': 0,  # 0: 전체, 1: 모바일 메인 언론사, 2: PC 메인 언론사
            'sort': 1,  # 0: 관련도순, 1: 최신순, 2: 오래된순
            'spq': 3, 
            'start': startNo,
            'where': 'news_tab_api',
            'nso': 'so:r,p:all,a:all'
        }
    )

    # JSON 형태의 문자열을 딕셔너리로 변환합니다.
    dat = json.loads(s = res.text)

    # 딕셔너리에서 HTML 형태의 문자열을 bs4.BeautifulSoup 객체로 변환합니다.
    soup = BTS(markup = dat['collection'][0]['html'], features = 'html.parser')

    # 네이버뉴스 링크를 포함하는 HTML 요소를 선택합니다.
    items = soup.select('div.info_group > a.info:nth-child(3)')

    # 네이버뉴스 링크를 시리즈로 생성합니다.
    links = pd.Series(data = [item['href'] for item in items])

    # 시리즈를 반환합니다.
    return links

In [ ]:
# 함수를 테스트합니다.
links = NaverNewsLink(searchWord = '백종원')

### 데이터 수집

In [ ]:
# 최종 결과를 저장할 빈 시리즈를 생성합니다.
newsLinks = pd.Series(dtype = str)

# 반복문 시작 위치를 설정합니다.
i = 1

# while 반복문으로 관련 네이버뉴스 링크를 수집합니다.(최대 1000개까지)
# [참고] 일간지 기사에서 네이버뉴스 링크가 없는 경우가 많습니다.
while i <= 1000:

    # 시작 위치를 출력합니다.
    print(i)

    # 네이버뉴스 링크를 수집하고 links에 할당합니다.
    links = NaverNewsLink(
        searchWord = '백종원', 
        period = 3, 
        bgnDate = '2025.01.01', 
        endDate = '2025.04.14', 
        startNo = i
    )

    # 새로 수집한 네이버뉴스 링크 개수를 n에 할당합니다. 
    n = len(links)

    # 새로 수집한 네이버뉴스 링크 개수가 0이면 반복문을 중단합니다.
    # 새로 수집한 네이버뉴스 링크와 직전에 수집한 네이버뉴스 링크가 일치하면 반복문을 중단합니다.
    # 두 시리즈가 일치하지 않으면 최종 결과에 새로 수집한 네이버뉴스 링크에 추가합니다.
    if n == 0:
        break
    elif links.equals(newsLinks.iloc[-n:]): 
        break
    else:
        newsLinks = pd.concat(objs = [newsLinks, links], ignore_index = True)

    # 시작 위치를 업데이트합니다.
    i += 10
    
    # 1초간 멈춥니다.
    time.sleep(1)

In [ ]:
# newsLinks의 처음 5행을 확인합니다.
newsLinks.head()

In [ ]:
# newsLinks의 정보를 확인합니다.
newsLinks.info()

### 데이터 전처리

In [ ]:
# newsLinks에서 쿼리 문자열 패턴이 있는지 확인합니다.
newsLinks.str.contains(pat = '(\?.+)').sum()

In [ ]:
# newsLinks에서 쿼리 문자열 패턴을 추출하고 도수를 확인합니다.
newsLinks.str.extractall(pat = '(\?.+)')[0].value_counts()

In [ ]:
# newsLinks에서 쿼리 문자열을 삭제하고 newsLinks에 재할당합니다.
newsLinks = newsLinks.str.replace(pat = '(\?.+)', repl = '', regex = True)

In [ ]:
# newsLinks에서 원소의 중복 여부를 dups로 생성합니다.
# [참고] keep 매개변수에 'first'를 지정하면 첫 번째 중복 건은 False로 반환합니다.
dups = newsLinks.duplicated(keep = 'first')

In [ ]:
# newsLinks에서 중복인 행을 삭제하고 newsLinks에 재할당합니다.
if dups.sum() > 0:
    newsLinks = newsLinks.loc[~dups]

In [ ]:
# newsLinks의 행 개수를 확인합니다.
newsLinks.shape[0]

### 외부 파일로 저장

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# newsLinks를 pkl 파일로 저장합니다.
pd.to_pickle(obj = newsLinks, filepath_or_buffer = 'Naver_News_Link.pkl')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

## End of Document